In [8]:
import eee

import pandas as pd
import numpy as np

from eee.ensemble import Ensemble
from eee.evolve.fitness import FitnessContainer
from eee.evolve.fitness import ff_on
from eee.evolve.fitness import ff_off
from eee.evolve.wright_fisher import wright_fisher


# Basic ensemble
ens = Ensemble(R=1)
ens.add_species(name="s1",
                dG0=0,
                observable=True)
ens.add_species(name="s2",
                dG0=0.167,
                observable=False,
                mu_stoich={"X":2})

mu_dict = {"X":[0,3.333,1]}

ddg_df = pd.DataFrame({"site":[1,1],
                       "mut":["A1V","A1P"],
                       "s1":[-1.677,0.167],
                       "s2":[3.333,-5000]})

fc = FitnessContainer(ens,
                      mu_dict,
                      [ff_on,ff_off,ff_on],
                      select_on="fx_obs",
                      fitness_kwargs={},
                      T=1)

gc = eee.evolve.genotype.GenotypeContainer(fc=fc,
                                           ddg_df=ddg_df)


In [9]:
X = {"a":1,"b":2}

next(iter(X))

'a'

In [10]:
ens._species["s1"]["mu_stoich"] = {"X":0}

z_matrix = np.zeros((len(ens.species),len(mu_dict[ens.mu_list[0]])),dtype=float)
obs_mask = np.zeros(len(ens.species),dtype=bool)

first_key = ens.mu_list[0]
for i, name in enumerate(ens._species):
    z_matrix[i,:] = ens._species[name]["dG0"]
    
    mu_stoich = ens._species[name]["mu_stoich"]
    for j in range(len(mu_dict[ens.mu_list[0]])):
        for mu in ens.mu_list:
            z_matrix[i,j] += mu_dict[mu][j]*mu_stoich[mu]

    obs_mask[i] = ens._species[name]["observable"]

not_obs_mask = np.logical_not(obs_mask)

#max_allowed = np.log(np.finfo('d').max)*0.9


In [11]:

beta = 1/(ens._R*298.15)

mut_energy = np.array([1,2])

weights = (z_matrix + mut_energy[:,None])*beta
shift = max_allowed - np.max(weights,axis=1)
weights = weights + shift[:,None]
weights = np.exp(weights)

obs = weights[obs_mask,:]
not_obs = weights[not_obs_mask,:]

not_obs/(not_obs + obs)



array([[0.49441076, 0.5       , 0.49608762]])

In [12]:


first_key = iter(next(self._stored_mu))
for i, name in enumerate(self.species):
    mu_stoich = self.species[name]["mu_stoich"]
    self._mu_matrix[i,:] = self.species[name]["dG0"]
    for j in range(len(self._stored_mu[first_key])):
        for mu in self._stored_mu:
            self._mu_matrix[i,j] += self._stored_mu[mu][j]*mu_stoich[mu]


NameError: name 'self' is not defined

In [13]:
ens.get_obs(mut_energy={},mu_dict=mu_dict,T=1)

,T,X,s1,s2,fx_obs,dG_obs
0,1.0,0.000,0.541653,0.458347,0.541653,-0.167
1,1.0,3.333,0.001503,0.998497,0.001503,6.499
2,1.0,1.000,0.137881,0.862119,0.137881,1.833


In [14]:
eee.evolve.fitness._fitness_function(ens=ens,
                                     mut_energy={},
                                     mu_dict=mu_dict,
                                     fitness_fcns=[ff_on,ff_off],
                                     select_on="fx_obs",
                                     fitness_kwargs={},
                                     T=1)

array([0.54165324, 0.99849732])

In [15]:

df = ens.get_obs(mu_dict=mu_dict,
                 T=1)
v = np.array(df["fx_obs"])
print(ff_on(v[0]),ff_off(v[1]))
df

0.5416532393676118 0.9984973180670926


,T,X,s1,s2,fx_obs,dG_obs
0,1.0,0.000,0.541653,0.458347,0.541653,-0.167
1,1.0,3.333,0.001503,0.998497,0.001503,6.499
2,1.0,1.000,0.137881,0.862119,0.137881,1.833


In [16]:
df = ens.get_obs(mu_dict=mu_dict,
                 mut_energy={"s1":-1.677,"s2":3.333},
                 T=1)
v = np.array(df["fx_obs"])
print(ff_on(v[0]),ff_off(v[1]))
df


0.9943867704232219 0.8159281306107955


,T,X,s1,s2,fx_obs,dG_obs
0,1.0,0.000,0.994387,0.005613,0.994387,-5.177
1,1.0,3.333,0.184072,0.815928,0.184072,1.489
2,1.0,1.000,0.959960,0.040040,0.959960,-3.177


In [17]:
df = ens.get_obs(mu_dict=mu_dict,
                 mut_energy={"s1":0.167,"s2":-16.667},
                 T=1)
v = np.array(df["fx_obs"])
print(ff_on(v[0]),ff_off(v[1]))
df


5.775822590594909e-08 0.999999999926446


,T,X,s1,s2,fx_obs,dG_obs
0,1.0,0.000,5.775823e-08,1.0,5.775823e-08,16.667
1,1.0,3.333,7.355409e-11,1.0,7.355409e-11,23.333
2,1.0,1.000,7.816726e-09,1.0,7.816726e-09,18.667


In [18]:
print(fc.fitness({}))
print(fc.fitness({"s1":-0.167,"s2":3.333}))
print(fc.fitness({"s1":0.167,"s2":-16.667}))

0.07457161348016993
0.7812798793868584
4.514802407097093e-16


In [19]:
gc, generations = wright_fisher(gc=gc,
                                mutation_rate=0.001,
                                population=10000,
                                num_generations=1000,)

  0%|          | 0/999 [00:00<?, ?it/s]

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/harmsm/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/p_/hcs03cdj48nbsvw72d92tr040000gn/T/ipykernel_65533/2645006398.py", line 1, in <module>
    gc, generations = wright_fisher(gc=gc,
  File "/Users/harmsm/work/programming/git-clones/eee/eee/evolve/wright_fisher.py", line 184, in wright_fisher
    new_index = gc.mutate(index=population[j])
  File "/Users/harmsm/work/programming/git-clones/eee/eee/evolve/genotype.py", line 296, in mutate
  File "/Users/harmsm/work/programming/git-clones/eee/eee/evolve/fitness.py", line 162, in fitness
    def fitness(self,mut_energy):
  File "/Users/harmsm/work/programming/git-clones/eee/eee/evolve/fitness.py", line 44, in _fitness_function
    T=T)
  File "/Users/harmsm/work/programming/git-clones/eee/eee/ensemble.py", line 370, in get_obs
ValueError: operands could not be broadc

In [20]:

genotypes = {}
for gen in generations:
    for g in gen:
        mut_genotype = "".join(gc.genotypes[g].mutations)
        if mut_genotype not in genotypes:
            genotypes[mut_genotype] = 0
        
        genotypes[mut_genotype] += gen[g]
        
assert genotypes["A1V"] > genotypes["A1P"]
print(genotypes)
        

NameError: name 'generations' is not defined

In [21]:
9821432

9821432

In [22]:
print(84213/100000)
print(15733/100000)
print(54/100000)

0.84213
0.15733
0.00054


In [23]:
31964/10000000

0.0031964

In [24]:

import numpy as np





In [25]:
# Four condition for two ligands
conditions = np.array([[1,2]])

# Stoich for three species and two ligands
stoich = np.array([[1,2,3],
                   [4,5,6]])

conditions.T*stoich


array([[ 1,  2,  3],
       [ 8, 10, 12]])

In [26]:
dG0 = np.array([0,0,0])

stoich = np.array([[1,0,0],
                   [0,1,0]])


dGmut = np.array([0,0,0])

